In [1]:
import scarches as sca
import scanpy as sc
import numpy as np

datasets = ["forebrain", "pancreas", "gastrulation_erythroid", "dentategyrus_lamanno_P5"]

# Specify the path to the dataset and model directory
for dataset in datasets:
    print(f"dataset: {dataset}")
    model = "expimap"
    model_path = f"{dataset}/model"

    # Load the saved EXPIMAP model
    adata_path = f"{dataset}/{model}_{dataset}.h5ad"
    adata = sc.read_h5ad(adata_path)

    # Assuming 'MuMs' and other necessary data are in `adata`
    data = adata.obsm["MuMs"]
    adata_expimap = sc.AnnData(X=data)
    adata_expimap.uns["terms"] = adata.uns["terms"].copy()
    adata_expimap.obs = adata.obs.copy()
    mask = adata.varm["I"]
    adata_expimap.varm["I"] = np.concatenate([mask, mask], axis=0)

    # Add a dummy 'study' column to adata_expimap.obs as required by EXPIMAP
    adata_expimap.obs["study"] = "0"

    # Load the saved EXPIMAP model
    intr_cvae = sca.models.EXPIMAP.load(model_path, adata=adata_expimap)
    adata.obsm['z'] = intr_cvae.get_latent(mean=False, only_active=False)

    print(f"building adata gp")
    # Now the model is loaded, and you can perform further analysis or use its methods
    z = adata.obsm["z"].copy()
    embedding_key_2d = "X_umap" if dataset == "dentategyrus_lamanno" else "X_umap"
    umap = adata.obsm[embedding_key_2d].copy() 
    obs = adata.obs.copy()
    var_names = adata.uns["terms"]
    gp_velocity = adata.obsm["gp_velo"].copy()
    adata_gp = sc.AnnData(X=z)
    adata_gp.obs = obs
    adata_gp.var_names = var_names
    adata_gp.obsm[embedding_key_2d] = umap
    adata_gp.layers["spliced"] = adata_gp.X
    adata_gp.layers["Ms"] = adata_gp.X
    adata_gp.layers["Mu"] = adata_gp.X
    adata_gp.uns = adata.uns.copy()
    adata_gp.write(f"{dataset}/adata_gp.h5ad")

/mnt/data2/home/leonardo/miniconda3/envs/DeepTrajectory/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/mnt/data2/home/leonardo/miniconda3/envs/DeepTrajectory/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/mnt/data2/home/leonardo/miniconda3/envs/DeepTrajectory/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


dataset: forebrain
AnnData object with n_obs × n_vars = 1720 × 4898
    obs: 'Clusters', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'isomap_1', 'isomap_2', 'isomap_3', 'isomap_1+2', 'isomap_1+3', 'isomap_2+3', 'pca_1', 'pca_2', 'pca_3', 'pca_1+2', 'pca_1+3', 'pca_2+3', 'isomap_1_ve', 'isomap_2_ve', 'isomap_3_ve', 'isomap_1_MuMs', 'isomap_2_MuMs', 'isomap_3_MuMs', 'isomap_1_z', 'isomap_2_z', 'isomap_3_z', 'isomap_1_velocity', 'isomap_2_velocity', 'isomap_3_velocity', 'isomap_1_velocity_concat', 'isomap_2_velocity_concat', 'isomap_3_velocity_concat', 'isomap_1_PCA', 'isomap_2_PCA', 'isomap_3_PCA', 'velocity_self_transition', 'velocity_length', 'velocity_confidence', 'velocity_confidence_transition', 'root_cells', 'end_points', 'velocity_pseudotime', 'directional_variance', 'directional_difference', 'directional_cosine_sim_variance', 'directional_cosine_sim_difference', 'directional_cosine_sim_mean', 'velocities_miVelo_0_self_transition', 'velocities_miV

KeyboardInterrupt: 

In [ ]:
import numpy as np

# Get the number of observations (cells) in the dataset
n_cells = adata_expimap.n_obs

# Define the fraction used for training
train_frac = 0.9

# Set the seed if one was used during training to ensure reproducibility
np.random.seed(2020)  # Seed used during training

# Generate a shuffled list of indices
all_indices = np.arange(n_cells)
np.random.shuffle(all_indices)

# Determine the number of training samples
n_train = int(n_cells * train_frac)

# Split the indices into training and testing sets
train_indices = all_indices[:n_train]
test_indices = all_indices[n_train:]

print(f"Training indices: {train_indices}")
print(f"Test indices: {test_indices}")


In [ ]:
import numpy as np
import torch

# Step 1: Extract the input data for the test indices
input_data_test = adata_expimap.X[test_indices, :]  # Convert to dense if necessary

# Step 2: Determine the device (CPU or GPU) from the model's parameters
device = next(intr_cvae.model.parameters()).device  # Check the device of the model's parameters

# Step 3: Use the model to reconstruct the data
# Pass the input data through the model's decoder
with torch.no_grad():  # Ensure the model doesn't compute gradients
    input_tensor = torch.tensor(input_data_test).float().to(device)  # Ensure tensor is on the correct device
    reconstructed_data_test = intr_cvae.model.decoder(input_tensor).cpu().numpy()  # Move the output back to CPU if necessary

# Step 4: Compute MSE across genes for each cell
mse_per_cell_test = np.mean((input_data_test - reconstructed_data_test) ** 2, axis=1)

# Print or store the MSE for further analysis
print("MSE per cell:", mse_per_cell_test)


In [ ]:
print(f"Input data shape: {input_data_test.shape}")
print(f"Model input dimension: {intr_cvae.model.input_dim}")


In [ ]:
adata.varm

In [ ]:
print(f"Mask shape: {adata.varm['I'].shape}")
print(f"Mask shape: {intr_cvae.model.mask.shape}")


In [ ]:
import pickle

# Load the model's attributes dictionary to see the expected conditions
with open(f"{model_path}/attr.pkl", "rb") as f:
    attr_dict = pickle.load(f)

# Print the expected conditions
print("Expected conditions:", attr_dict['conditions_'])


In [ ]:
import scarches as sca
import scanpy as sc
import numpy as np
import torch
import pandas as pd

# Load the saved EXPIMAP model
dataset = "forebrain"
model_path = f"{dataset}/model"
adata_path = f"{dataset}/expimap_{dataset}.h5ad"
adata = sc.read_h5ad(adata_path)

data = adata.obsm["MuMs"]
adata_expimap = sc.AnnData(X=data)
adata_expimap.uns["terms"] = adata.uns["terms"].copy()
adata_expimap.obs = adata.obs.copy()
mask = adata.varm["I"]
adata_expimap.varm["I"] = np.concatenate([mask, mask], axis=0)

# Initialize 'study' column in adata_expimap.obs to match expected condition
adata_expimap.obs["study"] = "0"  # Set to string '0' to match expected condition during training

# Ensure 'study' column is of string type
adata_expimap.obs["study"] = adata_expimap.obs["study"].astype(str)

# Load the model ensuring the condition is already set in adata
intr_cvae = sca.models.EXPIMAP.load(model_path, adata=adata_expimap)

# Get the test indices from your earlier split
n_cells = adata_expimap.n_obs
train_frac = 0.9
np.random.seed(2020)
all_indices = np.arange(n_cells)
np.random.shuffle(all_indices)
n_train = int(n_cells * train_frac)
train_indices = all_indices[:n_train]
test_indices = all_indices[n_train:]

# Extract test input data
input_data_test = adata_expimap.X[test_indices, :]

# Get the device
device = next(intr_cvae.model.parameters()).device

# Extract the condition labels for the test indices as a NumPy array
condition_test = adata_expimap.obs["study"].values[test_indices]

# Create a mapping from condition labels to indices
condition_to_label = {c: i for i, c in enumerate(intr_cvae.conditions_)}
# Map the condition labels in condition_test to their corresponding indices
condition_test_indices = np.array([condition_to_label[c] for c in condition_test])

# Forward pass using the latent variables
with torch.no_grad():
    # Get the latent representation by passing input data and condition labels
    z = intr_cvae.get_latent(x=input_data_test, c=condition_test)

    # Convert z to a PyTorch tensor and move it to the correct device
    z = torch.tensor(z).float().to(device)

    # Prepare the batch tensor for the decoder
    batch = torch.tensor(condition_test_indices).to(device)

    # Pass the latent variables to the decoder to get the reconstructed data
    reconstructed_data_test, _ = intr_cvae.model.decoder(z, batch=batch)
    reconstructed_data_test = reconstructed_data_test.cpu().numpy()

# Compute MSE across genes for each cell
mse_per_cell_test = np.mean((input_data_test - reconstructed_data_test) ** 2, axis=1)
print("MSE per cell:", mse_per_cell_test)


In [ ]:
adata.obs

In [ ]:
intr_cvae.trainer

In [ ]:
mse_per_cell_test.shape

In [ ]:
reconstructed_data_test

In [ ]:
print(f"Input data shape: {input_data_test.shape}")  # Should be (N, 4898)
print(f"Mask shape: {adata_expimap.varm['I'].shape}")  # Should align with features
print(f"Model input dimension: {intr_cvae.model.input_dim}")



In [ ]:
# Forward pass with the condition
with torch.no_grad():
    input_tensor = torch.tensor(input_data_test).float().to(device)
    output = intr_cvae.model(input_tensor, batch=condition_test)  # Pass the condition
    
    # Check what the output is by inspecting each element
    print(f"Output: {output}")

    # If we expect reconstructed data, let's inspect the individual components of the tuple
    for i, part in enumerate(output):
        print(f"Part {i} - shape: {part.shape if isinstance(part, torch.Tensor) else part}")


In [ ]:
# Forward pass using the latent variables
with torch.no_grad():
    # Assuming `get_latent()` encodes the input data
    z = intr_cvae.get_latent(input_data_test)  # Get latent representation
    
    # Now pass the latent variables to the decoder to get the reconstructed data
    reconstructed_data_test, _ = intr_cvae.model.decoder(z, batch=condition_test)
    reconstructed_data_test = reconstructed_data_test.cpu().numpy()

# Compute MSE across genes for each cell
mse_per_cell_test = np.mean((input_data_test - reconstructed_data_test) ** 2, axis=1)
print("MSE per cell:", mse_per_cell_test)
